<h3>Задание 7. Транспортная задача</h3><br>Вариант 6

In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import Bounds, linprog
np.set_printoptions(suppress=True)

In [2]:
A = np.array([270, 340, 290])
B = np.array([140, 80, 240, 310, 120])
C = np.array([[9, 14, 6, 5, 4],
              [13, 15, 12, 3, 7],
              [17, 10, 11, 8, 27]])

In [3]:
N = len(A)
M = len(B)

In [4]:
df = pd.DataFrame(C, columns=B, index=A)
df.columns.name = 'Ai \\ Bj'
df

Ai \ Bj,140,80,240,310,120
270,9,14,6,5,4
340,13,15,12,3,7
290,17,10,11,8,27


Допустимый план перевозки груза методом минимальной стоимости

In [5]:
P = np.zeros_like(C)
INF = 1_000_000_007
C_ = C.copy()
A_ = A.copy()
B_ = B.copy()
i = 0
j = 0
while not np.all(C_ == INF):
    i, j = np.unravel_index(C_.argmin(), C_.shape)
    P[i][j] = min(A_[i], B_[j])
    if A_[i] <= B_[j]:
        C_[i].fill(INF)
        B_[j] -= A_[i]
        A_[i] = 0
    else:
        C_[:, j].fill(INF)
        A_[i] -= B[j]
        B_[j] = 0    
if np.count_nonzero(P) == N + M - 1:
    print('Опорный план невырожденный')
else:
    print('Опорный план вырожденный')

F = 0
for i in range(N):
    for j in range(M):
        F += C[i][j] * P[i][j]
print('Полученный план перевозки груза:\n', P)
print('Значение целевой функции для этого опорного плана F =', F)

Опорный план является невырожденным
Полученный план перевозки груза:
 [[  0   0 150   0 120]
 [ 30   0   0 310   0]
 [-30  80  90   0   0]]
Значение целевой функции для этого опорного плана F = 3980


Оптимальный план перевозки груза

In [6]:
A_ub = np.array([[1 if i in range(j*M, M*(j+1)) else 0 for i in range(N*M)] for j in range(N)])
b_ub = A
b_eq = B
A_eq = np.array([[1 if i % M == j else 0 for i in range(N*M)] for j in range(M)])

res = linprog(C.flatten(), A_ub, b_ub, A_eq, b_eq, options={'disp': True})

X = res.x.round(3).reshape(N, M)
F_min = res.fun.round(3)

Primal Feasibility  Dual Feasibility    Duality Gap         Step             Path Parameter      Objective          
1.0                 1.0                 1.0                 -                1.0                 161.0               
0.1530082820257     0.1530082820257     0.1530082820257     0.8574814989773  0.1530082820257     803.5612438597      
0.1025126388982     0.1025126388982     0.1025126388982     0.3409116690241  0.1025126388983     1069.173735039      
0.0627623608946     0.0627623608946     0.06276236089461    0.4286785279633  0.06276236089464    1713.456845614      
0.01088668887341    0.0108866888735     0.01088668887351    0.8477308844445  0.01088668887315    3955.597336498      
0.008190541471482   0.008190541471552   0.008190541471553   0.2742268794766  0.008190541471287   4381.68860077       
0.001359295752282   0.001359295752316   0.001359295752316   0.8359114056213  0.001359295752357   5659.720138877      
1.811248636616e-05  1.811248636663e-05  1.811248636673e-0

In [7]:
print('Оптимальный план перевозки груза X*:\n', X)
print('Оптимальная стоимость перевозки:\nF_min = F(X*) =', F_min)

Оптимальный план перевозки груза X*:
 [[140.   0.  40.   0.  90.]
 [  0.   0.   0. 310.  30.]
 [  0.  80. 200.   0.   0.]]
Оптимальная стоимость перевозки:
F_min = F(X*) = 6000.0
